In [1]:
#Đọc dữ liệu
import pandas as pd
df = pd.read_csv("../../crawl_data/raw_data/phongvu.csv", on_bad_lines='skip')
print(df.shape)

df['Giá bán'] = pd.to_numeric(df['Giá bán'].str.replace(r'[^\d]', '', regex=True), errors='coerce')

df = df.rename(columns={
    'Tên sản phẩm':'Tên Sản Phẩm',
    'Giá bán':'Giá Bán',
    'Thương hiệu':'Hãng Sản Xuất',
    'Chip đồ họa':'GPU',
    'Lưu trữ':'Bộ Nhớ',
    'Hệ điều hành':'Hệ Điều Hành',
    'Pin':'Dung Lượng Pin',
    'Khối lượng':'Trọng Lượng',
    'Ram':'RAM',
    'Kích thước':'Độ Dày',
    'Cổng kết nối':'Cổng Giao Tiếp',
    'Màn hình':'Màn Hình',
})



(832, 35)


In [108]:
def print_if_null(df, target_column, condition_column):
    """
    In ra các giá trị trong target_column nếu condition_column bị thiếu (NaN).
    
    Parameters:
        df (pd.DataFrame): DataFrame chứa dữ liệu
        target_column (str): Cột cần in giá trị
        condition_column (str): Cột dùng để kiểm tra giá trị thiếu (NaN)
    """
    null_rows = df[df[condition_column].isna()]
    print(null_rows[target_column].dropna().tolist())  # loại bỏ NaN nếu có trong target_column


In [109]:
#Chuyển đổi RAM thành số
import re

def extract_total_ram(ram_string):
    if not isinstance(ram_string, str):
        return None  # hoặc 0 nếu bạn muốn xem như không có RAM
    ram_string = ram_string.upper()

    # Trường hợp: "2 x 32GB", "2x16GB"
    match_multi = re.search(r'(\d+)\s*[xX]\s*(\d+)\s*GB', ram_string)
    if match_multi:
        count = int(match_multi.group(1))
        size = int(match_multi.group(2))
        return count * size

    # Trường hợp cộng nhiều thanh RAM như: "8GB (4GB + 4GB)"
    match_sum = re.findall(r'(\d+)\s*GB', ram_string)
    if match_sum:
        sizes = list(map(int, match_sum))
        return sum(sizes)

    # Trường hợp chỉ có một dung lượng: "16GB Onboard", "32GB LPDDR5x"
    match_single = re.search(r'(\d+)\s*GB', ram_string)
    if match_single:
        return int(match_single.group(1))

    return ram_string
df['RAM'] = df['RAM'].apply(extract_total_ram)

In [110]:
#Lấp đầy RAM trống
import pandas as pd
import numpy as np

def fill_missing_ram(df, name_column='Tên Sản Phẩm', ram_column='RAM'):
    # Khởi tạo bản đồ ánh xạ
    ram_mapping = {
        'MacBook Air M1': 8,
        'MacBook Air M2': 8,
        'MacBook Air M3': 8,
        'MacBook Air M4': 8,
        'MacBook Pro M2': 8,
        'MacBook Pro M3': 8,
        'MacBook Pro M4': 18,
        'MacBook Pro M4 Pro': 18,
        'MacBook Pro M4 Max': 36,
        'MacBook Pro M2 Max': 32,
        'MacBook Pro M2 Pro': 16,
        'MacBook Air Core i5': 8,
         'MacBook Pro 14 inch M3': 8,
    'MacBook Pro 14 inch M3 Max': 36,
    'MacBook Pro 14 inch M2 Pro': 16,
    'MacBook Pro 14 inch M2 Max': 32,
    'Macbook Pro 14 inch M2 Max': 32,  # bản viết hoa khác
    'MacBook Pro 16 inch M2 Max': 32,
    'MacBook Pro 16 inch M2 Pro': 16,
    'Apple MacBook Air Apple M2': 8,
    'Máy tính xách tay Macbook Air': 8,
    }

    # Hàm ánh xạ từng dòng
    def map_ram(row):
        if pd.notna(row[ram_column]):
            return row[ram_column]

        for key in ram_mapping:
            if isinstance(row[name_column], str) and key in row[name_column]:
                return ram_mapping[key]
        return np.nan  # Nếu không khớp thì để NaN

    # Gán lại cột RAM
    df[ram_column] = df.apply(map_ram, axis=1)
    return df
df = fill_missing_ram(df)
print(df['RAM'].unique())

[ 96.  64.  16.  88.  32.  12.  24.  72.   8. 128.  48.  56.  18.  44.
  28.  36.  nan   4.  40. 192.]


In [111]:
missing_count = df['RAM'].isna().sum()
print(f"Số giá trị thiếu: {missing_count}")

Số giá trị thiếu: 3


In [112]:
#Chuyển đổi PIN thành số
import pandas as pd
import re

# Hàm trích xuất dung lượng pin
def extract_battery_size(battery_string):
    if pd.isna(battery_string):
        return None
    battery_string = str(battery_string)  # Đảm bảo là chuỗi
    match = re.search(r'(\d+\.?\d*)\s*(Wh|Whrs|Whs|whr|wH)?', battery_string, re.IGNORECASE)
    if match:
        return float(match.group(1))
    return None

# Áp dụng chuẩn hóa cột Dung Lượng Pin
df['Dung Lượng Pin'] = df['Dung Lượng Pin'].apply(extract_battery_size)

# Tính giá trị trung bình cho các giá trị hợp lệ (≥ 10)
mean_valid_battery = df.loc[df['Dung Lượng Pin'] >= 10, 'Dung Lượng Pin'].mean()

# Thay thế các giá trị < 10 bằng giá trị trung bình hợp lệ
df.loc[df['Dung Lượng Pin'] < 10, 'Dung Lượng Pin'] = mean_valid_battery

In [113]:
#Lấp các pin trống
import re

def map_battery_capacity(product_name):
    mapping = {
        r"MacBook Air.*M1.*13": 49.9,
        r"MacBook Air.*M[234].*13": 52.6,
        r"MacBook Air.*M[234].*15": 66.5,
        r"MacBook Pro.*M2.*13": 58.2,
        r"MacBook Pro.*M[234].*14": 70.0,
        r"MacBook Pro.*M[234].*16": 100.0,
        r"MacBook Air.*Core i5": 54.0,
        r"MacBook Pro.*M4 Max.*16": 100.0,
        r"MacBook Pro.*M4 Pro.*14": 70.0,
        r"Máy tính xách tay Macbook Air.*13": 52.6,
        r".*": None  # mặc định
    }

    if not isinstance(product_name, str):
        return None

    for pattern, value in mapping.items():
        if re.search(pattern, product_name, flags=re.IGNORECASE):
            return value
    return None

# Ví dụ áp dụng cho df
df.loc[df["Dung Lượng Pin"].isna(), "Dung Lượng Pin"] = df.loc[df["Dung Lượng Pin"].isna(), "Tên Sản Phẩm"].apply(map_battery_capacity)

print(df['Dung Lượng Pin'].unique())


[ 60.55769231  50.          60.          65.          75.
  52.6         66.5         70.          57.          58.
  47.          52.          53.          54.          45.
  73.                  nan  72.          77.          99.
  80.          36.          56.         100.          76.
  40.          90.          58.2         48.          49.9
  49.        ]


In [114]:
missing_count = df['Dung Lượng Pin'].isna().sum()
print(f"Số giá trị thiếu: {missing_count}")

Số giá trị thiếu: 30


In [115]:
#Chuyển đổi CPU thành i3,i5,m3,m5,ryzen3.ryzen5,...
import re
import pandas as pd

def clean_string(s):
    # Loại bỏ dấu đặc biệt ® ™
    s = re.sub(r'[®™]', '', s)
    # Loại bỏ các dấu ngoặc và bên trong ngoặc
    s = re.sub(r'\(.*?\)', '', s)
    # Loại bỏ chữ số GHz, số cache, số lõi, thread,...
    s = re.sub(r'\d+(\.\d+)?\s*ghz', '', s)
    s = re.sub(r'\d+\s*mb', '', s)
    s = re.sub(r'\d+\s*(cores?|threads?|luồng|lõi|cache)', '', s)
    s = re.sub(r'[^\w\s-]', ' ', s)  # Thay các ký tự đặc biệt còn lại thành khoảng trắng
    s = re.sub(r'\s+', ' ', s)  # Chuẩn hóa khoảng trắng
    return s.strip().lower()

def extract_cpu_type(cpu_string):
    if pd.isna(cpu_string):
        return None

    s = clean_string(cpu_string)

    # Apple chip (m1,m2,m3,m4...)
    apple_match = re.search(r'\bm(\d+)\b', s)
    if 'apple' in s or apple_match:
        if apple_match:
            return f"M{apple_match.group(1).upper()}"
        else:
            return "Apple Unknown"
    
    # Ryzen
    ryzen_match = re.search(r'ryzen\s*(ai\s*)?r?(\d)', s)
    if ryzen_match:
        return f"Ryzen {ryzen_match.group(2)}"
    
    # Intel Core i3/i5/i7/i9
    intel_i_match = re.search(r'\b(i[3579])\b', s)
    if intel_i_match:
        return intel_i_match.group(1).upper()
    
    # Intel Core + số, ví dụ core 5, core 7
    intel_core_num_match = re.search(r'core\s*(?:i[3579])?[-\s]?(\d)', s)
    if intel_core_num_match:
        return f"Core {intel_core_num_match.group(1)}"
    
    # Core + số chung (dùng để bắt core 1, core 5...)
    core_num_match = re.search(r'core\s+(\d)', s)
    if core_num_match:
        return f"Core {core_num_match.group(1)}"

    # Ultra + số, có thể là Ultra U7-xxx, lấy số đầu tiên sau ultra
    ultra_match = re.search(r'ultra\s*[u]?(\d)', s)
    if ultra_match:
        return f"Ultra {ultra_match.group(1)}"
    
    # Snapdragon / Qualcomm
    if 'qualcomm' in s or 'snapdragon' in s:
        return 'Qualcomm Snapdragon'
    
    # AMD (nếu không rõ kiểu Ryzen)
    if 'amd' in s:
        return 'AMD'
    
    # Celeron (Intel thấp cấp)
    if 'celeron' in s:
        return 'Celeron'
    
    # Nếu không khớp
    return None



# Áp dụng cho cột CPU
df['Hãng CPU'] = df['CPU'].apply(extract_cpu_type)
print(df['Hãng CPU'].unique())


['Ultra 9' 'I5' 'Ryzen 5' 'Ultra 5' 'Ryzen 9' 'Ultra 7' 'I9' 'Core 7' 'I3'
 'I7' 'Qualcomm Snapdragon' 'M2' 'M3' 'Core 5' 'M4' 'Ryzen 7' 'AMD' None
 'M1']


In [116]:
missing_count = df['Hãng CPU'].isna().sum()
print(f"Số giá trị thiếu: {missing_count}")

Số giá trị thiếu: 3


In [117]:
#Tách số Lõi
import re

def extract_cores(cpu_str):
    if not cpu_str or not isinstance(cpu_str, str):
        return None
    
    # 1. Tìm dạng (X lõi / Y luồng) hoặc X lõi / Y luồng
    match_lu_thread = re.search(r'(\d+)\s*lõi\s*/\s*\d+\s*luồng', cpu_str, re.I)
    if match_lu_thread:
        return int(match_lu_thread.group(1))
    
    # 2. Tìm tổng hợp P-core, E-core, LPE-core
    p_cores = sum(int(x) for x in re.findall(r'(\d+)\s*P[-\s]?core', cpu_str, re.I)) or 0
    e_cores = sum(int(x) for x in re.findall(r'(\d+)\s*E[-\s]?core', cpu_str, re.I)) or 0
    lpe_cores = sum(int(x) for x in re.findall(r'(\d+)\s*LPE[-\s]?core', cpu_str, re.I)) or 0
    total = p_cores + e_cores + lpe_cores
    if total > 0:
        return total
    
    # 3. Tìm dạng "X lõi"
    match_core_only = re.search(r'(\d+)\s*lõi', cpu_str, re.I)
    if match_core_only:
        return int(match_core_only.group(1))
    
    # 4. Tìm dạng "XC" hoặc "X C"
    match_c = re.search(r'(\d+)\s*C', cpu_str, re.I)
    if match_c:
        return int(match_c.group(1))
    
    # 5. Tìm dạng "X cores", "X core", "X nhân"
    match_core_words = re.search(r'(\d+)\s*(core|cores|nhân)', cpu_str, re.I)
    if match_core_words:
        return int(match_core_words.group(1))
    
    # --- Bắt đầu dự đoán số lõi nếu không tìm thấy rõ ---
    
    cpu_str_lower = cpu_str.lower()
    
    # 6. Dự đoán lõi cho Intel theo dòng và series
    # i9 thường 14-24 lõi (mình lấy trung bình 20)
    if 'i9' in cpu_str_lower:
        if 'hx' in cpu_str_lower:
            return 24  # thường 24 lõi (ví dụ 13900HX)
        elif 'h' in cpu_str_lower:
            return 14  # 14 lõi phổ biến H-series i9
        else:
            return 10  # các dòng i9 khác (U, G...)
    
    # i7 phổ biến 8-14 lõi
    if 'i7' in cpu_str_lower:
        if 'hx' in cpu_str_lower:
            return 16  # i7 HX series thường 16 lõi (vd 13650HX)
        elif 'h' in cpu_str_lower:
            return 14  # i7 H-series phổ biến 14 lõi (vd 13700H)
        elif 'u' in cpu_str_lower:
            return 10  # i7 U-series có thể 10 lõi (vd 13620U)
        else:
            return 8
    
    # i5 phổ biến 4-10 lõi
    if 'i5' in cpu_str_lower:
        if 'hx' in cpu_str_lower:
            return 12  # i5 HX thường 12 lõi
        elif 'h' in cpu_str_lower:
            return 8
        elif 'u' in cpu_str_lower:
            return 6
        else:
            return 4
    
    # i3 thường 2-4 lõi
    if 'i3' in cpu_str_lower:
        if 'h' in cpu_str_lower or 'hx' in cpu_str_lower:
            return 6
        else:
            return 4
    
    # Intel Ultra (Core Ultra 7/5...) thường 8-16 lõi, tính trung bình 12 lõi
    if 'ultra' in cpu_str_lower:
        return 12
    
    # AMD Ryzen 7 ~ 8 lõi, Ryzen 5 ~ 6 lõi, Ryzen 3 ~ 4 lõi
    if 'ryzen' in cpu_str_lower:
        if '7' in cpu_str_lower:
            return 8
        if '5' in cpu_str_lower:
            return 6
        if '3' in cpu_str_lower:
            return 4
    
    # AMD Ryzen AI 7 thường ~ 8 lõi
    if 'ryzen ai 7' in cpu_str_lower:
        return 8
    
    # Nếu có "core" nhưng không rõ, lấy 4 làm mặc định
    if 'core' in cpu_str_lower:
        return 4
    
    cpu_str_lower = cpu_str.lower()
    
    # Snapdragon (Qualcomm)
    if 'snapdragon' in cpu_str_lower:
        # Nếu tìm thấy số lõi dạng 'X lõi' thì ưu tiên dùng
        match_core = re.search(r'(\d+)\s*lõi', cpu_str_lower)
        if match_core:
            return int(match_core.group(1))
        
        # Nếu có số trong chuỗi, nhưng không rõ định dạng
        # Có thể thử lấy số cache hoặc số lớn nhất gần đó (tạm thời không)
        
        # Dự đoán theo dòng phổ biến:
        # Snapdragon cao cấp X Plus, X Elite thường 8 lõi
        if 'x plus' in cpu_str_lower or 'x elite' in cpu_str_lower:
            return 8
        
        # Nếu có X1E, X1P hoặc X1 thì giả định 8 lõi
        if 'x1e' in cpu_str_lower or 'x1p' in cpu_str_lower or 'x1' in cpu_str_lower:
            return 8
        
        # Mặc định Snapdragon khác lấy 8 lõi
        return 8

    # Không đoán được trả None
    return np.nan


df['Số Lõi'] = df['CPU'].apply(extract_cores)
print(df['Số Lõi'].unique())

[24.  8.  6. 12. 16. 14. 10.  4.  5. 20. nan  2.]


In [118]:
missing_count = df['Số Lõi'].isna().sum()
print(f"Số giá trị thiếu trong cột 'Số Lõi': {missing_count}")

Số giá trị thiếu trong cột 'Số Lõi': 4


In [119]:
#Tách xung nhịp tối đa
import re
import numpy as np
def clean_cpu_string(cpu_str):
    if not cpu_str or not isinstance(cpu_str, str):
        return ""
    
    # Loại bỏ các ký tự đặc biệt như ™ ® ( ) [ ] dấu nháy đơn, kép,...
    cleaned = re.sub(r'[™®\(\)\[\]\'\"]', '', cpu_str)
    
    # Thay thế dấu gạch ngang dài/dạng Unicode thành dấu '-'
    cleaned = re.sub(r'[–—−]', '-', cleaned)
    
    # Thay nhiều khoảng trắng thành 1 khoảng trắng
    cleaned = re.sub(r'\s+', ' ', cleaned)
    
    # Xóa khoảng trắng đầu cuối
    cleaned = cleaned.strip()
    
    # Chuyển thành chữ thường
    cleaned = cleaned.lower()
    
    return cleaned

# Bảng tra cứu model CPU (có thể mở rộng thêm)
cpu_freq_lookup = {
    'intel core i5-12450h': 4.4,
    'intel core i5-13420h': 4.6,
    'intel core i9-14900hx': 5.4,
    'intel core ultra 5-125u': 4.7,
    'intel core i5-1334u': 4.5,
    'intel core i7-1355u': 5.0,
    'amd ryzen 9 hx 370': 4.8,
    'amd ryzen 7 7435hs': 4.3,
    'intel core ultra 7-155u': 5.2,
    'intel core i7-12650h': 4.7,
    'amd ryzen 7 6800h': 4.7,
    'intel core i5-1135g7': 4.2,
    'intel core i7-13620h': 5.0,
    'apple m2': 3.5,
    'apple m3 pro': 3.7,
    'apple m4 pro': 3.9,
    'amd ryzen ai 9 hx 370': 4.0,
    'amd ryzen 7 7435hs': 4.0,
    'intel core 7-150u': 5.0,
    'intel core i7-150u': 5.0,
    'intel core ultra 7 155h': 5.6,
    'intel core i7-1355u': 5.0,
    'intel core i5-1035g4': 3.7,
    'intel core i5-11320h': 4.4,
    'apple m3': 3.7,
    'intel core i3-n305': 3.0,
    'alder lake i5-12450h': 4.4,
    'intel core ultra u7-155h': 5.6,
    'apple m3 max': 3.9,
    'amd ryzen 7-5700u': 4.3,
    'intel core i5-1235u': 4.4,
    'amd ryzen r7-8845hs': 4.3,
    'intel core i5-1240p': 4.4,
    'core i5-1335u': 4.6,
    'intel core i7-14650hx': 5.4,
    'core i7-13700h': 5.4,
    'intel core i5-1135g7': 4.2,
    'intel core i5 tiger lake - 1135g7': 4.2,
    'amd ryzen ai 9 hx 370': 4.0,
    'amd ryzen 7 7435hs': 4.0,
    'intel core 7-150u': 5.0,
    'intel core i7-150u': 5.0,
    'intel core ultra 7 155h': 5.6,
    'intel core i7-1355u': 5.0,
    'intel core i5-1035g4': 4.1,
    'intel core i5-1135g7': 4.2,
    'core 7-150u': 5.0,
    'intel core i7-14650hx': 5.4,
    'intel core i5 1135g7': 4.2,
    'cpu i7-1355u': 5.0,
    'amd ryzen ai 9 hx 370': 4.0,
    'amd ryzen 7 7435hs': 4.0,
    'intel core 7-150u': 5.0,
    'intel core i7-150u': 5.0,
    'intel core ultra 7 155h': 5.6,
    'intel core i7-1355u': 5.0,
    'intel core i5-1035g4': 4.1,
    'intel core i7-14650hx series': 5.4,
    '5,00 ghz':5.0,
    '4.7 mhz':4.7
}

def safe_float(val):
    try:
        return float(val)
    except:
        return None

def extract_max_frequency(cpu_strg):
    cpu_str = clean_cpu_string(cpu_strg)
    if not cpu_str or not isinstance(cpu_str, str):
        return None
    
    cpu_str_lower = cpu_str.lower()
    
    # 1. Tìm tất cả số kèm "GHz"
    ghz_numbers = re.findall(r'([\d\.]+)\s*ghz', cpu_str_lower)
    ghz_floats = [safe_float(n) for n in ghz_numbers if safe_float(n) is not None]
    if ghz_floats:
        max_ghz = max(ghz_floats)
        if 1.0 <= max_ghz <= 6.0:
            return max_ghz
    
    # 2. Tìm số gần các từ khóa max turbo, up to,... không kèm GHz
    freq_candidates = re.findall(r'(?:max turbo frequency|max turbo|turbo boost|up to|tối đa|max)\D*([\d\.]+)', cpu_str_lower)
    freq_candidates_floats = [safe_float(n) for n in freq_candidates if safe_float(n) is not None]
    filtered = [f for f in freq_candidates_floats if 1.0 <= f <= 6.0]
    if filtered:
        return max(filtered)
    
    # 3. Tìm tất cả số thập phân có thể, lọc trong khoảng hợp lý
    numbers = re.findall(r'[\s\(,]([\d\.]+)[\s\),]', cpu_str_lower)
    numbers_float = [safe_float(n) for n in numbers if safe_float(n) is not None]
    filtered_numbers = [f for f in numbers_float if 1.0 <= f <= 6.0]
    if filtered_numbers:
        return max(filtered_numbers)
    
    # 4. Nếu không tìm thấy số, thử tra lookup dựa trên tên model CPU
    for model_key in cpu_freq_lookup:
        if model_key in cpu_str_lower:
            return cpu_freq_lookup[model_key]
    
    # 5. Không tìm thấy gì, trả về None
    return np.nan

# Áp dụng cho cột CPU trong dataframe
df['Xung nhịp tối đa'] = df['CPU'].apply(extract_max_frequency)
print(df['Xung nhịp tối đa'].unique())


[5.5  4.6  4.4  5.   5.4  4.5  5.1  4.8  4.55 5.2  4.3  4.7  4.9  4.2
 3.4  3.5  3.7   nan 4.75 5.8  2.97 3.9  2.6  3.8  4.1  4.   5.3 ]


In [120]:
missing_count = df['Xung nhịp tối đa'].isna().sum()
print(f"Số giá trị thiếu trong cột 'Xung nhịp tối đa': {missing_count}")

Số giá trị thiếu trong cột 'Xung nhịp tối đa': 16


In [121]:
#Tách Cache
import re
# Mapping ước lượng cache MB theo model hoặc keyword đơn giản
CACHE_MAPPING = {
    'i3-1215u': 10,
    'i5-13420h': 12,
    'i7-14700hx': 30,
    'i9-14900hx': 36,
    'ryzen 5 7535hs': 16,
    'ryzen 7 7435hs': 16,
    'ryzen 5 8640hs': 20,
    'ryzen 7 7840hs': 20,
    'intel core ultra 7 155h': 24,
    'core i5 1235u': 12,
    'ultra 7 155h': 24,
    'ultra 5 125u': 12,
    'core i7 13700h': 30,
    'intel core i7-13620h': 24,
    'core i5 1334u': 12,
    'cpu i7-1355u': 24,
    'amd ryzen r5-7535hs': 16,
    'amd ryzen ai 9 hx 370': 24,
    'core i3 1315u': 10,
    'core i7 1355u': 24,
    'intel core 7 150u': 24,
    'intel core 7-1255u': 24,
    'ultra 7 155u': 24,
    'core 7 150u': 24,
    'intel core i5-1335u': 12,
    'intel core i5-1334u': 12,
    'intel core ultra 9 185h': 30,
    'intel core 7-150u': 24,
    'intel core i7-150u': 24,
    'intel core i3-1305u': 10,
    'intel core i7-1355u': 24,
    'intel core i7-12650h': 24,
    'intel core i7-13700h': 30,
    'intel core i5-12450h': 12,
    'intel core i7-1335u': 24,
    'intel core i7-13620h': 24,
    'intel core i7-1255u': 24,
    'amd ryzen 5-5600h': 16,
    'intel core i7-i7-14650hx series': 30,
    'intel core i5 1135g7': 12,
    'intel core i5 tiger lake 1135g7': 12,
    'intel core i5-1035g4': 8,
    'intel core i5-1135g7': 8,
    'intel core i5-11320h': 12,
    'intel core i5-1235u': 12,
    'intel core i5-13500h': 24,
    'cpu 10 lõi': 24,
    'apple m4 10 lõi': 24,
    'apple m4 pro 12 lõi': 30,
    'intel core i3-n305': 6,
    'intel celeron n4120': 4,
    'apple m4 max 16 lõi': 36,
    'intel core ultra 5 125h': 12,
    'alder lake i5-12450h': 12,
    'intel core ultra u7-155h': 24,
    'core i3 1215u': 6,
    'intel core 5 120u': 24,
    'apple m3 pro 12 nhân': 30,
    'core 7-150u': 24,
    'cpu 28 lõi': 36,
    'intel core i5-1240p': 12,
    'core i7-13700h': 30,
    'apple m3 8 lõi': 24,
    'apple m3 pro 16 nhân': 30,
    'apple m2 pro 10 nhân': 24,
    'intel core i5 tiger lake 1135g7': 8,
  'AMD Ryzen 7 6800H': 20,
  'Intel CoreTM i5-1235U': 12,
  'Apple M2 8 nhân': 24,
  'Apple M2 8 lõi': 24,
  'Apple M4 8 lõi': 24,
  'Apple M3 Max 14 nhân': 30,
  'Intel Core Ultra 5 - 125H': 12,
  'Intel Core i7 - 1335U': 12,
  'Apple M4 Pro 14 lõi': 30,
  'AMD Ryzen 7-5700U': 8,
  'Chip Apple M4 Max 14 lõi': 36,
  'AMD Ryzen R7-8845HS': 20,
  'Intel Core i5 Tiger Lake - 1135G7': 8,
  '16GB': 16,
}

def clean_cpu_string(cpu_str):
    if not cpu_str or not isinstance(cpu_str, str):
        return ""
    cleaned = re.sub(r'[™®\(\)\[\]\'\"/,]', ' ', cpu_str)  
    cleaned = re.sub(r'[–—−]', '-', cleaned)
    cleaned = re.sub(r'\s+', ' ', cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.lower()
    return cleaned

def extract_cache(cpu_str):
    if not cpu_str or not isinstance(cpu_str, str):
        return None

    cpu_str_clean = clean_cpu_string(cpu_str)

    # Tìm cache trong chuỗi
    cache_matches = re.findall(r'(\d{1,3})\s*m[b]?\b', cpu_str_clean)
    caches = []
    for c in cache_matches:
        try:
            val = int(c)
            if 3 <= val <= 200:
                caches.append(val)
        except:
            continue

    if caches:
        return max(caches)

    # Nếu không tìm thấy cache trong chuỗi, thử đoán dựa trên mapping
    for key in CACHE_MAPPING:
        if key in cpu_str_clean:
            return CACHE_MAPPING[key]

    # Nếu không đoán được, trả None
    return None

df['Cache'] = df['CPU'].apply(extract_cache)


In [122]:
import re
import numpy as np

def map_cache(name):
    if not isinstance(name, str):
        return None
    
    mapping = {
        r"MacBook Pro.*14.*M3.*": 70.0,
        r"MacBook Pro.*14.*M3 Max.*": 75.0,
        r"Apple MacBook Pro.*M4.*14\.2\"": 70.0,
        r"Apple MacBook Pro.*M4 Pro.*16\.2\"": 100.0,
        r"Apple MacBook Pro.*M4 Max.*14\.2\"": 75.0,
        r"MacBook Air.*M2.*13.*": 52.6,
        r"MacBook Air.*M2.*15.*": 66.5,
        r"MacBook Air.*M3.*13.*": 52.6,
        r"MacBook Air.*M3.*15.*": 66.5,
        r"MacBook Air.*M4.*13.*": 52.6,
        r"MacBook Air.*M4.*15.*": 66.5,
        r"MacBook Pro.*16.*M2 Max.*": 100.0,
        r"MacBook Pro.*16.*M2 Pro.*": 100.0,
        r"MacBook Pro.*14.*M2 Pro.*": 70.0,
        r"MacBook Pro.*14.*M2 Max.*": 75.0,
        r"MacBook Pro.*16.*M2 Pro.*": 100.0,
        r"MacBook Pro.*M2.*13.*": 58.2,
        r"MacBook Air.*Core i5.*13\.3\"": 50.0,
        r"Máy tính xách tay Macbook Air.*13\.6\"": 52.6,
        # Thêm mẫu cho MacBook Air/MacBook Pro với các năm hoặc biến thể khác
        r"MacBook Air M2 2022.*": 52.6,
        r"MacBook Air M2 2023.*": 52.6,
        r"MacBook Air M3 2024.*": 52.6,
        r"MacBook Pro M2 2022.*13.*": 58.2,
        r"MacBook Pro M3 2023.*14.*": 70.0,
        r"MacBook Pro M2 Pro 2023.*": 70.0,
        r"MacBook Pro M2 Max 2023.*": 75.0,
        r"MacBook Pro M4 14\.2\"": 70.0,
        r"MacBook Pro M4 Pro 16\.2\"": 100.0,
        r"MacBook Pro M4 Max 14\.2\"": 75.0,
        # Bạn có thể thêm nhiều mẫu regex hơn theo mẫu trên
    }

    for pattern, cache_value in mapping.items():
        if re.search(pattern, name, flags=re.IGNORECASE):
            return cache_value
    return None
df.loc[df['Cache'].isna(), 'Cache'] = df.loc[df['Cache'].isna(), 'Tên Sản Phẩm'].apply(map_cache)
print(df['Cache'].unique())

[ 36.   12.   16.    8.   24.   10.   20.   30.   18.   52.6  66.5  33.
  70.    4.   64.    nan 100.   42.    6.   58.2  50.   75. ]


In [123]:
missing_count = df['Cache'].isna().sum()
print(f"Số giá trị thiếu trong cột 'Cache': {missing_count}")

Số giá trị thiếu trong cột 'Cache': 7


In [124]:
#Chuyển Hệ điều hành
import pandas as pd
import re

def normalize_text(text):
    """Hàm chuẩn hóa văn bản: xóa ký tự đặc biệt, viết thường."""
    text = text.lower()
    text = re.sub(r'[^\w\s]', ' ', text)  # loại bỏ ký tự đặc biệt
    text = re.sub(r'\s+', ' ', text).strip()  # loại bỏ khoảng trắng dư
    return text

def extract_os_name(os_string):
    """Hàm phân loại hệ điều hành thành 5 nhóm."""
    if pd.isna(os_string):
        return None

    os_string = normalize_text(os_string)

    # Nhóm "No OS", FreeDOS, không cài sẵn,...
    if any(keyword in os_string for keyword in ['no os', 'noos', 'no os', 'free os', 'freedos', 'free operating system']):
        return 'No OS'

    # Nhóm MacOS
    if any(keyword in os_string for keyword in ['mac', 'macos', 'os x', 'big sur']):
        return 'MacOS'

    # Nhóm Ubuntu hoặc Linux
    if any(keyword in os_string for keyword in ['ubuntu', 'linux']):
        return 'Ubuntu'

    # Nhóm Windows 11
    if any(keyword in os_string for keyword in ['windows 11', 'win11', 'win 11', 'win11sl', 'win 11 home', 'windows 11 home']):
        return 'Win 11'

    # Nhóm Windows 10
    if any(keyword in os_string for keyword in ['windows 10', 'win10', 'win 10', 'windows 10 home', 'windows 10 pro']):
        return 'Win 10'

    return None

# Áp dụng hàm chuẩn hóa
df['Hệ Điều Hành'] = df['Hệ Điều Hành'].apply(extract_os_name)

print(df['Hệ Điều Hành'].unique())

['Win 11' 'MacOS' None 'No OS' 'Ubuntu' 'Win 10']


In [125]:
#Chuyển về bộ nhớ số
import re
import pandas as pd
import numpy as np

def extract_memory_capacity(text):
    if pd.isna(text):
        return np.nan
    
    text = str(text).lower().strip()
    
    # Nếu text không chứa chữ số, trả NaN
    if not re.search(r'\d', text):
        return np.nan
    
    # Kiểm tra RAM
    if any(keyword in text for keyword in ['ram', 'ddr', 'sodimm', 'lpddr', 'onboard', 'soldered']):
        # Tìm tất cả các số lượng GB hoặc TB có thể theo dạng (2x8gb), (1x16gb), 8gb ...
        matches = re.findall(r'(\d+(\.\d+)?)\s*gb', text)
        total_gb = 0
        if matches:
            # Cộng tổng dung lượng, có thể có nhiều nhóm
            for match in matches:
                total_gb += float(match[0])
            return total_gb
        
        # Nếu không có GB, thử tìm TB (hiếm RAM TB)
        matches_tb = re.findall(r'(\d+(\.\d+)?)\s*tb', text)
        if matches_tb:
            total_gb = 0
            for match in matches_tb:
                total_gb += float(match[0]) * 1024
            return total_gb
        
        # Nếu không tìm được gì, trả NaN
        return np.nan
    
    # Nếu không phải RAM, coi như Storage
    # Tìm dung lượng TB trước
    match_tb = re.search(r'(\d+(\.\d+)?)\s*tb', text)
    if match_tb:
        return float(match_tb.group(1)) * 1024  # TB → GB
    
    # Tìm dung lượng GB
    match_gb = re.search(r'(\d+(\.\d+)?)\s*gb', text)
    if match_gb:
        return float(match_gb.group(1))
    
    # Nếu text chỉ là số (ví dụ '512', '1024'), coi là GB
    if re.fullmatch(r'\d+', text):
        return float(text)
    
    # Trả NaN nếu không tìm được dung lượng rõ ràng
    return np.nan




# Áp dụng vào DataFrame
df['Bộ Nhớ'] = df['Bộ Nhớ'].apply(extract_memory_capacity)



In [126]:
#Lấp đầy bộ nhớ
import re
import pandas as pd
import numpy as np

def assign_storage_gb(name, current_storage):
    # Nếu current_storage đã có giá trị hợp lệ (không trống, không 0, không nan), giữ nguyên
    if current_storage not in [None, '', 0] and not pd.isna(current_storage):
        return current_storage
    
    if not isinstance(name, str):
        return current_storage

    name = name.lower()

    storage_mapping = {
 r"macbook air.*m1.*13": 256,
    r"macbook air.*m2.*13": 256,
    r"macbook air.*m2.*15": 512,
    r"macbook air.*m3.*13": 256,
    r"macbook air.*m3.*15": 512,
    r"macbook air.*m4.*13": 256,
    r"macbook air.*m4.*15": 512,

    r"macbook pro.*m1.*13": 256,
    r"macbook pro.*m2.*13": 512,
    r"macbook pro.*m3.*14": 512,
    r"macbook pro.*m3 max.*14": 1024,
    r"macbook pro.*m4.*14": 512,
    r"macbook pro.*m4 pro.*16": 1024,
    r"macbook pro.*m4 max.*16": 2048,
    r"macbook pro.*m2 pro.*14": 512,
    r"macbook pro.*m2 max.*14": 1024,
    r"macbook pro.*m2 pro.*16": 1024,
    r"macbook pro.*m2 max.*16": 2048,
    r"macbook pro.*m2.*13": 256,
    r"macbook pro 14 inch m3 2023": 512,
    r"macbook pro 14 inch m3 max 2023": 1024,
    r"macbook pro 16 inch m2 max 2023": 2048,
    r"macbook pro 16 inch m2 pro 2023": 1024,
    r"macbook pro 14 inch m2 pro 2023": 512,
    r"macbook pro 14 inch m2 max 2023": 1024,

    r"macbook air core i5.*13\.3": 128,
    r"máy tính xách tay macbook air.*13\.6": 256,
    }

    for pattern, storage_gb in storage_mapping.items():
        if re.search(pattern, name):
            return storage_gb
    
    # Nếu không match pattern, giữ nguyên giá trị hiện tại
    return current_storage

# Áp dụng với DataFrame, cột 'Tên sản phẩm' và cột 'Bộ nhớ'
df['Bộ Nhớ'] = df.apply(lambda row: assign_storage_gb(row['Tên Sản Phẩm'], row['Bộ Nhớ']), axis=1)

print(df['Bộ Nhớ'].unique())


[6144.  512. 1024. 2048.  256. 4096.   nan  128.]


In [127]:
missing_count = df['Bộ Nhớ'].isna().sum()
print(f"Số giá trị thiếu trong cột 'Bộ Nhớ': {missing_count}")

Số giá trị thiếu trong cột 'Bộ Nhớ': 3


In [128]:
#Chuyển đổi Màn Hình về loại màn hình
import pandas as pd
import re

def extract_panel_type(text):
    if pd.isna(text):
        return None
    
    text_orig = text  # lưu lại bản gốc
    
    text = text.lower()
    
    multi_word_panels = ['led', 'quantum dot']
    panel_keywords = ['oled', 'ips', 'wva', 'led', 'va', 'pls', 'amled', 'retina', 'tn']
    panel_variants_mapping = {
        'ips-level': 'IPS',
        'led-backlit': 'LED',
        'led backlit': 'LED',
        'oled display': 'OLED',
        'mini-led': 'LED',
        'micro-led': 'MICRO LED',
        'quantum-dot': 'QUANTUM DOT',
        'oled panel': 'OLED',
        'tft lcd': 'TFT/LCD',
        'led tft lcd': 'LED/TFT/LCD',
        'led/tft/lcd': 'LED/TFT/LCD',
    }
    
    text = re.sub(r"[\'\"]", " ", text)
    text = re.sub(r"[\-\_]", " ", text)
    text = re.sub(r"\s+", " ", text)
    
    for k, v in panel_variants_mapping.items():
        text = text.replace(k, v.lower())
    
    segments = re.split(r"[\/,;()]+", text)
    
    found_panels = []
    
    for seg in segments:
        seg = seg.strip()
        for multi in multi_word_panels:
            if multi in seg:
                if multi.upper() not in found_panels:
                    found_panels.append(multi.upper())
                seg = seg.replace(multi, '')
        tokens = re.findall(r'\b[a-z]+\b', seg)
        for token in tokens:
            if (token in [k.lower() for k in panel_keywords] or
                token.upper() in [v.upper() for v in panel_variants_mapping.values()]):
                if token.upper() not in found_panels:
                    found_panels.append(token.upper())
    
    # Nếu đã tìm thấy panel rõ ràng thì trả về
    if found_panels:
        return '/'.join(found_panels)
    
    # Nếu không tìm thấy panel, dự đoán dựa trên từ khóa phổ biến về độ phân giải hoặc thuật ngữ khác
    # Các từ khóa gợi ý thường là IPS (do phổ biến)
    ips_hint_keywords = ['fhd', 'wuxga', 'wqxga', 'qhd', '2k', '144hz', '165hz', '250nits', '300nits', 'anti glare', 'srgb', 'dci p3']
    
    for hint in ips_hint_keywords:
        if hint in text:
            return 'IPS'
    
    # Nếu vẫn không tìm được gì, gán 'UNKNOWN'
    return None



# Áp dụng vào cột "Màn hình"
df['Loại Màn Hình'] = df['Màn Hình'].apply(extract_panel_type)
print(df['Loại Màn Hình'].unique())

['LED/IPS' 'IPS' 'LED' None 'RETINA' 'WVA' 'TN']


In [129]:
missing_count = df['Loại Màn Hình'].isna().sum()
print(f"Số giá trị thiếu trong cột 'Màn Hình': {missing_count}")

Số giá trị thiếu trong cột 'Màn Hình': 43


In [130]:
#Tách độ phân giải màn hình
import re
import numpy as np
import pandas as pd

def extract_resolution(res_text):
    if pd.isna(res_text):
        return np.nan
    
    text = str(res_text).lower()
    
    match = re.search(r'(\d{3,5})\s*[x×]\s*(\d{3,5})', text)
    if match:
        width = int(match.group(1))
        height = int(match.group(2))
        return f"{width}x{height}"
    
    return np.nan

def map_resolution_keywords(text):
    if pd.isna(text):
        return np.nan
    
    text = str(text).lower()
    
    resolution_map = {
        'fhd+': '1920x1200',
        'fhd': '1920x1080',
        'qhd+': '2560x1600',
        'qhd': '2560x1440',
        'uhd+': '3840x2400',
        'uhd': '3840x2160',
        '4k': '3840x2160',
        '2k': '2560x1440',
        '8k': '7680x4320',
        'wuxga': '1920x1200',
        'wqhd': '2560x1440',
        'wxga': '1366x768',
        'hd': '1366x768',
        'hd+': '1600x900',
    }
    
    for k, v in resolution_map.items():
        if k in text:
            return v
    
    return np.nan  # chưa tìm thấy

# Dictionary chứa các mẫu mô tả => độ phân giải dự đoán
predict_map = {
    '2880x1880': '2880x1880',
    '2880x1620': '2880x1620',
    '1080x1920': '1080x1920',
    '14.0inch oled touch': '2048x1280',  # ví dụ gán 2048x1280
    '2048x1280': '2048x1280',
    '3000x2000': '3000x2000',
    '1920x1280': '1920x1280',
    '3072x1920': '3072x1920',
    '3840x2160': '3840x2160',
    '18 inch wqxga 240hz dci-p3': '2560x1600',
    '16 inch wqxga 240hz dci-p3': '2560x1600',
    '2736x1824': '2736x1824',
    # bạn có thể bổ sung thêm mapping chính xác hơn nếu cần
}

def extract_screen_resolution(text):
    # Cố gắng lấy độ phân giải dạng số
    res = extract_resolution(text)
    if pd.notna(res):
        return res
    
    # Nếu không có, thử lấy theo từ khóa phổ biến
    res_kw = map_resolution_keywords(text)
    if pd.notna(res_kw):
        return res_kw
    
    # Nếu vẫn không có, dò trong dict dự đoán
    if pd.isna(text):
        return np.nan
    text_lower = str(text).lower()
    
    for key in predict_map.keys():
        if key in text_lower:
            return predict_map[key]
    
    # Không tìm được, trả về unknown hoặc text gốc
    return np.nan  # hoặc return text_lower

# Áp dụng lên dataframe
df['Độ Phân Giải'] = df['Màn Hình'].apply(extract_screen_resolution)
print(df['Độ Phân Giải'].unique())


['3840x2400' '3840x1800' '1920x1080' '1920x1200' '2560x1600' '2880x1800'
 '3072x1920' '3200x2000' '2240x1400' '2560x1664' '2880x1864' '3024x1964'
 '2880x1620' '2800x1800' '2800x1620' '2560x1440' nan '3456x2234'
 '2048x1280' '1366x768' '3000x2000']


In [131]:
missing_count = df['Độ Phân Giải'].isna().sum()
print(f"Số giá trị thiếu: {missing_count}")

Số giá trị thiếu: 3


In [132]:
#Chuyển đổi cổng giao tiếp sang số lượng cổng giao tiếp
import pandas as pd
import re

# Các từ khóa nhóm chuẩn
patterns = {
    "USB-A": r"(USB[\s\-]?3(?:\.|,)?[01]?(?: Gen \d)?(?: Type[-\s]?A)?|USB Standard[-\s]?A)",
    "USB-C": r"(USB Type[-\s]?C|USB-C|USB4|Thunderbolt \d)",
    "HDMI": r"HDMI",
    "Ethernet": r"(RJ[-\s]?45|Ethernet|LAN port)",
    "Audio Combo": r"(Audio combo|headphone.*combo|3\.5mm|headset)",
    "SD Card Reader": r"(SD card|microSD)",
    "DisplayPort": r"(DisplayPort|Mini DisplayPort)",
    "Power Connector": r"(DC-in|AC adapter|power connector)",
    "Security Lock": r"(Security Lock|Kensington Lock)"
}

# Hàm trích xuất các loại cổng
def extract_ports(port_str):
    if pd.isna(port_str):
        return ""
    found_ports = []
    for label, pattern in patterns.items():
        if re.search(pattern, port_str, re.IGNORECASE):
            found_ports.append(label)
    return ", ".join(found_ports)

# Hàm đếm số lượng cổng
def count_ports(port_str):
    if pd.isna(port_str) or port_str.strip() == "":
        return 0
    return len(port_str.split(', '))

# Xử lý cổng giao tiếp
df["Loại Cổng Giao Tiếp"] = df["Cổng Giao Tiếp"].apply(extract_ports)
df["Số Cổng Giao Tiếp"] = df["Loại Cổng Giao Tiếp"].apply(count_ports)

# Xoá cột gốc
df.drop(columns=["Cổng Giao Tiếp"], inplace=True)
df.drop(columns=["Loại Cổng Giao Tiếp"], inplace=True)

print(df['Số Cổng Giao Tiếp'].unique())


[5 4 6 1 3 2 0]


In [133]:
missing_count = df['Số Cổng Giao Tiếp'].isna().sum()
print(f"Số giá trị thiếu trong cột Số Cổng Giao Tiếp: {missing_count}")

Số giá trị thiếu trong cột Số Cổng Giao Tiếp: 0


In [134]:
#Tách hãng GPU
import pandas as pd
import re
import unicodedata

def normalize_text(text):
    if pd.isna(text):
        return ""
    text = str(text)
    # chuẩn hóa unicode, loại bỏ dấu
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii')
    text = text.lower()

    # Sửa lỗi phổ biến typo (thêm các kiểu sai sót hay gặp)
    text = re.sub(r'rtxtm|rtx[\s\-]?tm', 'rtx', text)
    text = re.sub(r'geforce[\s\-]?rtx', 'rtx', text)  # gom geForce rtx thành rtx
    text = re.sub(r'nvidia[\s\-]?geforce', 'nvidia', text)
    
    # Loại bỏ các từ, cụm từ không cần thiết
    remove_words = [
        r'boost clock \d+mhz', r'tgp \d+w', r'ai tops \d+', r'gddr\d+', r'lapotp gpu', r'with \d+ gb of dedicated', 
        r'up to \d+mhz', r'maximum graphics power with dynamic boost', r'rog boost len en \d+mhz', r'100w', r'45w',
        r'6gb', r'8gb', r'4gb', r'16gb', r'24gb', r'10 loi', r'gpu \d+ loi',
        r'vram', r'cuda cores', r'gddr6', r'gddr5', r'gddr4'
    ]
    for w in remove_words:
        text = re.sub(w, '', text)

    # Loại bỏ ký tự đặc biệt thừa, nhiều khoảng trắng
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

def clean_gpu_brand_only(text):
    if pd.isna(text) or not str(text).strip():
        return None

    text = normalize_text(text)

    # Bỏ qua các chuỗi không liên quan
    ignore_keywords = [
        'ssd', 'nvme', 'neural engine', 'dong', 'nhan', 'loi', 'ram', 'ddr', 'mhz',
        'storage', 'memory', 'cpu', 'top', 'gen4', 'gen5', 'up to', 'gbps', 'pci', 'cong', 'toc do',
        'core', 'engine'
    ]
    if any(k in text for k in ignore_keywords):
        gpu_keywords = ['intel', 'nvidia', 'amd', 'radeon', 'adreno', 'arc', 'mx', 'rtx']
        if not any(gpu in text for gpu in gpu_keywords):
            return None

    if "intel" in text or "iris" in text or "uhd" in text or "arc" in text:
        return "Intel"

    if "nvidia" in text or "rtx" in text or "mx" in text:
        return "NVIDIA"

    if "amd" in text or "radeon" in text:
        return "AMD"

    if "adreno" in text:
        return "Qualcomm"

    return None


# Áp dụng vào DataFrame:
df["Hãng GPU"] = df["GPU"].apply(clean_gpu_brand_only)


In [135]:
gpu_mapping = {
    "ROG STRIX": "NVIDIA",
    "VIVOBOOK": "Intel",
    "IDEAPAD SLIM 5 OLED": "AMD",
    "LOQ": "NVIDIA",
    "ASPIRE 5": "Intel",
        "ROG STRIX": "NVIDIA",
    "TUF GAMING": "NVIDIA",
    "ZENBOOK": "Intel",
    "VIVOBOOK": "Intel",
    "PRESTIGE": "NVIDIA",
    "KATANA": "NVIDIA",
    "CYBORG": "NVIDIA",
    "MACBOOK": "Apple",
    "THINKPAD": "Intel",
    "IDEAPAD GAMING": "NVIDIA",
    'ASUS ROG': 'NVIDIA',
    'ASUS TUF': 'NVIDIA',
    'ASUS EXPERTBOOK': 'Intel',
    'ASUS VIVOBOOK': 'Intel',  # có thể NVIDIA tùy model cao

    'MSI THIN': 'NVIDIA',
    'MSI MODERN': 'Intel',
    'MSI': 'NVIDIA',  # fallback

    'HP ELITEBOOK': 'Intel',
    'HP PAVILION': 'Intel',
    'HP': 'Intel',

    'DELL INSPIRON': 'Intel',  # hoặc NVIDIA nếu model cao
    'DELL XPS': 'Intel',
    'DELL': 'Intel',

    'ACER ASPIRE LITE': 'Intel',
    'ACER ASPIRE': 'Intel',
    'ACER': 'Intel',

    'LENOVO THINKPAD': 'Intel',
    'LENOVO IDEAPAD': 'Intel',
    'LENOVO LOQ': 'NVIDIA',
    'LENOVO LEGION': 'NVIDIA',
    'LENOVO': 'Intel',

    'MACBOOK': 'Intel',  # hoặc Apple GPU / M1 M2 nếu có
    # ASUS
    'ASUS ROG': 'NVIDIA',
    'ASUS TUF': 'NVIDIA',
    'ASUS EXPERTBOOK': 'Intel',
    'ASUS VIVOBOOK': 'Intel',
    'ASUS K3607VJ': 'NVIDIA',
    'ASUS B1502CVA': 'Intel',

    # MSI
    'MSI THIN': 'NVIDIA',
    'MSI MODERN': 'Intel',
    'MSI': 'NVIDIA',

    # HP
    'HP ELITEBOOK': 'Intel',
    'HP PAVILION': 'Intel',
    'HP': 'Intel',

    # Dell
    'DELL INSPIRON': 'Intel',
    'DELL XPS': 'Intel',
    'DELL': 'Intel',

    # Acer
    'ACER ASPIRE LITE': 'Intel',
    'ACER ASPIRE': 'Intel',
    'ACER': 'Intel',

    # Lenovo
    'LENOVO THINKPAD': 'Intel',
    'LENOVO IDEAPAD': 'Intel',
    'LENOVO LOQ': 'NVIDIA',
    'LENOVO LEGION': 'NVIDIA',
    'LENOVO': 'Intel',

    # Apple – bổ sung mới
    'MAC MINI M4 PRO': 'Apple GPU',
    'MAC MINI M4': 'Apple GPU',
    'IMAC M4': 'Apple GPU',
    'MACBOOK AIR M1': 'Apple GPU',
    'MACBOOK AIR M2': 'Apple GPU',
    'MACBOOK PRO M3': 'Apple GPU',
    'MACBOOK': 'Apple GPU',
}

def map_gpu_brand(row):
    current = row.get('Hãng GPU', None)
    if pd.notna(current) and current != '':
        return current

    name = str(row.get('Tên Sản Phẩm', '')).upper()
    for keyword, brand in gpu_mapping.items():
        if keyword in name:
            return brand
    return None

df['Hãng GPU'] = df.apply(map_gpu_brand, axis=1)
print(df['Hãng GPU'].unique())



['Intel' 'NVIDIA' 'AMD' 'Qualcomm' 'Apple GPU' None]


In [136]:
missing_count = df['Hãng GPU'].isna().sum()
print(f"Số giá trị thiếu trong cột Hãng GPU: {missing_count}")

Số giá trị thiếu trong cột Hãng GPU: 3


In [137]:
#Chuyển đổi cột Độ Dày
import re
import numpy as np
import pandas as pd

def parse_thickness_advanced(value):
    if pd.isna(value):
        return np.nan
    
    # Nếu là số luôn
    if isinstance(value, (int, float)):
        val = float(value)
        if 10 <= val <= 50:
            return round(val, 2)
        elif val > 50:
            # Có thể sai đơn vị, chia 10
            val = val / 10
            if 10 <= val <= 50:
                return round(val, 2)
        elif val < 10:
            # Có thể sai đơn vị, nhân 10
            val = val * 10
            if 10 <= val <= 50:
                return round(val, 2)
        return np.nan
    
    s = str(value).lower()
    s = s.replace(',', '.')  # chuẩn hóa dấu thập phân
    s = s.replace('–', '-').replace('—', '-')  # chuẩn hóa dấu gạch ngang
    s = re.sub(r'\([^)]*\)', '', s)  # loại bỏ ngoặc đơn
    s = s.strip()
    
    # Tìm các phần có nhãn height, thickness, depth, dày, chiều dày
    labels = {
        'height': r'height[: ]*([\d\.]+)',
        'thickness': r'thickness[: ]*([\d\.]+)',
        'depth': r'depth[: ]*([\d\.]+)',
        'dày': r'dày[: ]*([\d\.]+)',
        'chiều dày': r'chiều dày[: ]*([\d\.]+)',
    }
    
    for label, pattern in labels.items():
        match = re.search(pattern, s)
        if match:
            val = float(match.group(1))
            # Xử lý đơn vị (cm hoặc mm)
            unit = 'cm' if 'cm' in s else 'mm'
            if unit == 'cm':
                val *= 10
            # Điều chỉnh theo quy tắc
            if 10 <= val <= 50:
                return round(val, 2)
            elif val > 50:
                val = val / 10
                if 10 <= val <= 50:
                    return round(val, 2)
            elif val < 10:
                val = val * 10
                if 10 <= val <= 50:
                    return round(val, 2)
            return np.nan
    
    # Tách chuỗi theo dấu x hoặc ×
    parts = re.split(r'[x×]', s)
    parts = [p.strip() for p in parts if p.strip()]
    
    if len(parts) >= 3:
        thickness_str = parts[-1]
        nums = re.findall(r'\d*\.?\d+', thickness_str)  # sửa ở đây
        if not nums:
            return np.nan
        nums = [float(n) for n in nums]
        avg_thickness = np.mean(nums)
        
        unit = 'cm' if 'cm' in s else 'mm'
        if unit == 'cm':
            avg_thickness *= 10
        
        # Áp dụng quy tắc điều chỉnh
        if 10 <= avg_thickness <= 50:
            return round(avg_thickness, 2)
        elif avg_thickness > 50:
            avg_thickness /= 10
            if 10 <= avg_thickness <= 50:
                return round(avg_thickness, 2)
        elif avg_thickness < 10:
            avg_thickness *= 10
            if 10 <= avg_thickness <= 50:
                return round(avg_thickness, 2)
        return np.nan
    
    # Xử lý dạng range hoặc nhiều số rải rác trong chuỗi
    nums = re.findall(r'\d*\.?\d+', s)  # sửa ở đây
    if not nums:
        return np.nan
    nums = [float(n) for n in nums]
    
    # Tìm số phù hợp sau điều chỉnh
    candidates = []
    for n in nums:
        for val in [n, n/10, n*10]:
            if 10 <= val <= 50:
                candidates.append(val)
    if not candidates:
        return np.nan
    thickness_val = min(candidates)
    return round(thickness_val, 2)

df['Độ Dày'] = df['Độ Dày'].apply(parse_thickness_advanced)

In [138]:
#Lấp đầy độ dày
thickness_mapping = {
    "Laptop Asus Zenbook S 14 UX5406SA-PV140WS": 10.9,
    "Laptop Gigabyte AORUS 16X ASG-53VNC94SH": 20.8,
    "Laptop Gigabyte G6 KF-H3VN853KH": 23.9,
    "Laptop Gigabyte G6 MF-H2VN854KH": 23.9,
    "Apple MacBook Air M2 13.6\"": 11.3,
    "Apple MacBook Air M3 13.6\"": 11.3,
    "Apple MacBook Air M4 13.6\"": 11.3,
    "Apple MacBook Air M4 15.3\"": 11.5,
    "Apple MacBook Air M2 2022 13 inch": 11.3,
    "MacBook Air M2 2022 13 inch": 11.3,
    "MacBook Air M3 2024 13 inch": 11.3,
    "MacBook Air M3 13 inch": 11.3,
    "MacBook Air M3 2024 15 inch": 11.5,
    "MacBook Air M2 2023 15 inch": 11.5,
    "MacBook Air M1 2020 13 inch": 16.1,
    "Apple MacBook Air Core i5 13.3\"": 17.0,
    "Macbook Air 13.6\"": 11.3,
    "Laptop Dell Inspiron 15 3530 - 71053721": 17.5,
    "Laptop Dell Inspiron 15 3530 - P16WD": 17.5,
    "Laptop Dell Inspiron 15 3530 - P16WD2": 17.5,
    "Laptop HP Elitebook 840 G8  (i7-1165G7/ Onboard graphics/ 16GB/ 512GB/": 17.9,
    "Apple MacBook Pro M4 Pro 14.2\"": 15.5,
    "Apple MacBook Pro M4 Max 14.2\"": 15.5,
    "Apple MacBook Pro M4 Max 16.2\"": 16.8,
    "Apple MacBook Pro M4 Pro 16.2\"": 16.8,
    "MacBook Pro 14 inch M3 2023": 15.5,
    "MacBook Pro 14 inch M2 Pro 2023": 15.5,
    "MacBook Pro 14 inch M2 Max 2023": 15.5,
    "Macbook Pro 14 inch M2 Max 2023": 15.5,
    "MacBook Pro 16 inch M2 Max 2023": 16.8,
    "MacBook Pro 16 inch M2 Pro 2023": 16.8,
    "MacBook Pro M2 2022 13 inch": 15.6,
    "MacBook Pro M2 2022 13 inch": 15.6,
    "Apple MacBook Pro M4 14.2\"": 15.5,
    "Laptop Lenovo Legion 5 16IRX9 - 83DG0051VN": 22.0,
}
def fill_missing_thickness(df):
    """
    Gán giá trị độ dày cho các dòng laptop nếu cột 'Độ Dày' đang bị thiếu (NaN),
    dựa theo tên laptop.
    """
    df["Độ Dày"] = df.apply(
        lambda row: thickness_mapping.get(row["Tên Sản Phẩm"], row["Độ Dày"])
        if pd.isna(row["Độ Dày"]) else row["Độ Dày"],
        axis=1
    )
    return df
df = fill_missing_thickness(df)

print(df['Độ Dày'].unique())

[28.02 24.9  22.9  22.7  25.1  25.38 12.82 19.95 10.9  15.65 17.9  20.21
 18.4  19.9  14.9  23.5  21.94 19.7  20.   15.9  16.15 16.1  18.9  17.99
 30.   20.8  23.9  16.9  17.4  14.3  11.3  18.6  11.5  26.8  27.75 15.45
 15.3  21.42 15.6  26.9  12.4  18.44 19.34 23.65 19.05 14.65 21.04 26.5
 25.15 18.95 19.2  17.8  17.   14.96 24.75 18.99 17.5  11.   19.64 15.95
 19.85 20.45 15.5  16.6  23.56 21.7  32.   15.7  12.6  16.48 16.   30.5
 20.85 19.37 23.4  22.42 23.92 27.45 17.31 22.75 24.95 22.62 17.25 22.
 21.9  24.47 22.45   nan 18.   25.9  17.47 17.1  19.   19.35 25.19 13.9
 26.95 15.49 24.85 16.8  17.12 20.9  17.37 11.35 21.1  27.9  27.4  25.83
 23.33 18.2  28.4  11.77 20.65 13.45 28.15 23.   25.55 23.25 16.4  19.3
 26.   24.4  22.65 22.5  15.98 23.42 24.05 24.28 25.95 25.02 17.42 24.03
 21.2  23.76]


In [139]:
#Chuyển đổi cột trọng lượng
import pandas as pd
import numpy as np

def clean_weight_column(df, col='Trọng Lượng'):
    def parse_weight(x):
        if pd.isna(x):
            return np.nan
        try:
            # Nếu đã là số thì trả luôn
            if isinstance(x, (int, float)):
                return float(x)
            # Chuẩn hóa string
            s = str(x).lower().replace(' ', '').replace(',', '.').replace('kg', '')
            # Nếu sau khi loại bỏ 'kg' còn là số thì trả về float
            val = float(s)
            return val
        except:
            return np.nan  # Không chuyển được thì NaN

    df['Trọng Lượng'] = df['Trọng Lượng'].apply(parse_weight)
    return df

df = clean_weight_column(df)

In [140]:
import pandas as pd
import re

# Map regex pattern -> trọng lượng (kg)
macbook_weight_map = {
    r"macbook air.*m1.*13": 1.29,
    r"macbook air.*m2.*13": 1.24,
    r"macbook air.*m2.*15": 1.51,
    r"macbook air.*m3.*13": 1.24,
    r"macbook air.*m3.*15": 1.51,
    r"macbook air.*m4.*13": 1.24,
    r"macbook air.*m4.*15": 1.51,
    r"macbook air.*core i5.*13": 1.35,
    r"macbook air.*13": 1.29,

    r"macbook pro.*m2.*13": 1.40,
    r"macbook pro.*m2.*14": 1.60,
    r"macbook pro.*m2.*16": 2.15,

    r"macbook pro.*m3.*13": 1.40,
    r"macbook pro.*m3.*14": 1.60,
    r"macbook pro.*m3.*16": 2.15,

    r"macbook pro.*m4.*14.*max": 1.63,
    r"macbook pro.*m4.*14.*pro": 1.60,
    r"macbook pro.*m4.*14": 1.60,
    r"macbook pro.*m4.*16.*max": 2.16,
    r"macbook pro.*m4.*16.*pro": 2.14,
    r"macbook pro.*m4.*16": 2.15,

    r"macbook pro.*14": 1.60,
    r"macbook pro.*16": 2.15,
    r"macbook pro.*13": 1.40,

    r"hp elitebook 840 g8": 1.32,
}

# Hàm ánh xạ trọng lượng từ tên máy
def get_weight_from_name(name):
    if not isinstance(name, str):
        return None
    name = name.lower()
    for pattern, weight in macbook_weight_map.items():
        if re.search(pattern, name):
            return weight
    return None

# Lấp đầy cột 'Trọng Lượng' nếu còn thiếu
df['Trọng Lượng'] = df.apply(
    lambda row: get_weight_from_name(row['Tên']) if pd.isnull(row['Trọng Lượng']) else row['Trọng Lượng'],
    axis=1
)

# Kiểm tra kết quả
print(df['Trọng Lượng'].unique())


[3.6  2.1  1.9  2.   2.5  2.7  1.2  1.85 1.7  1.5  2.3  2.9  1.4  1.37
 2.4  1.6  1.3  1.29  nan 2.8  3.3  1.8  0.9  2.2  1.1  1.49 1.66 1.24
 1.27 1.12 1.38 2.6  1.   3.   3.1  1.63 2.15 1.35 3.5 ]


In [141]:
missing_count = df['Trọng Lượng'].isna().sum()
print(f"Số giá trị bị thiếu: {missing_count}")

Số giá trị bị thiếu: 16


In [142]:
print_if_null(df, target_column='Tên Sản Phẩm', condition_column='Trọng Lượng')

['Apple MacBook Air M4 15.3"', 'Apple MacBook Air M4 15.3"', 'Apple MacBook Air M4 15.3"', 'MacBook Air M3 15 inch 2024', 'MacBook Air M3 15 inch 2024', 'MacBook Air M3 2024 15 inch', 'MacBook Air M3 2024 15 inch', 'MacBook Air M2 2023 15 inch', 'MacBook Air M2 2023 15 inch', 'MacBook Air M2 2023 15 inch', 'MacBook Air M2 2023 15 inch', 'Laptop HP Elitebook 840 G8  (i7-1165G7/ Onboard graphics/ 16GB/ 512GB/', 'Apple MacBook Air Core i5 13.3"']


In [143]:
selected_features = [
    'RAM',
    'Bộ Nhớ',
    'Dung Lượng Pin',
    'Giá Bán',
    'Số Lõi',
    'Xung nhịp tối đa',
    'Cache',
    'Số Cổng Giao Tiếp',
    'Hệ Điều Hành',
    'Hãng CPU',
    'Hãng GPU',
    'Loại Màn Hình',
    'Độ Phân Giải',
    'Độ Dày',
    'Trọng Lượng',
    'Hãng Sản Xuất',
    'Tên Sản Phẩm'
]

df1 = df[selected_features].copy()


In [144]:
print(df1.columns)

Index(['RAM', 'Bộ Nhớ', 'Dung Lượng Pin', 'Giá Bán', 'Số Lõi',
       'Xung nhịp tối đa', 'Cache', 'Số Cổng Giao Tiếp', 'Hệ Điều Hành',
       'Hãng CPU', 'Hãng GPU', 'Loại Màn Hình', 'Độ Phân Giải', 'Độ Dày',
       'Trọng Lượng', 'Hãng Sản Xuất', 'Tên Sản Phẩm'],
      dtype='object')


In [145]:
df1.to_csv("../clean_data_test/clean_phongvu.csv", index=False)